In [ ]:
!pip install -U pip
!pip install japanize-matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import pandas_profiling

%matplotlib inline

In [ ]:
!cat ../input/data_description.txt

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error


In [18]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sample_submission = pd.read_csv('../input/sample_submission.csv')

In [19]:
sample_submission.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [ ]:
pandas_profiling.ProfileReport(train)

In [ ]:
pandas_profiling.ProfileReport(test)

In [21]:
submission = test.loc[:, ['Id']]

X_train = train.drop(['SalePrice', 'Id'], axis=1)
y_train = train.loc[:, ['SalePrice']]

X_train = X_train.select_dtypes(include=np.number)
X_test = test.loc[:, X_train.columns]

In [22]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(test)


In [23]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [24]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20
                #valid_sets=lgb_eval,
                #early_stopping_rounds=5
               )

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
#print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

Saving model...
Starting predicting...


In [26]:
submission['SalePrice'] = y_pred
submission.to_csv('submission_lightgbm_not_using_sklearn_IF.csv', index=False)